# **Chapter 17: Representation Learning and Generative Learning Using Autoencoders and GANs**

## **1. Pendahuluan**

Bayangkan jika kita bisa belajar mengenali wajah seseorang tanpa pernah diberi tahu nama atau label apa pun. Atau, bayangkan jika komputer bisa "bermimpi" dan menciptakan gambar pemandangan yang belum pernah ada sebelumnya. Inilah dunia **Unsupervised Learning** dan **Generative Learning**.

Dalam bab-bab sebelumnya, kita sebagian besar berfokus pada *Supervised Learning* (ada label target). Di bab ini, kita akan menjelajahi dua arsitektur Deep Learning yang bekerja tanpa label:
1.  **Autoencoders:** Jaringan saraf yang belajar menyalin input ke output dengan cara yang efisien. Tujuannya adalah mempelajari representasi (coding) yang ringkas dari data.
2.  **Generative Adversarial Networks (GANs):** Dua jaringan saraf yang saling bersaing—satu mencoba memalsukan data, yang lain mencoba mendeteksi kepalsuan tersebut. Hasilnya adalah kemampuan untuk menghasilkan data baru yang sangat realistis.

Mengapa ini penting?
* **Dimensionality Reduction:** Mengompresi data tanpa kehilangan informasi vital.
* **Feature Extraction:** Mempelajari fitur berguna untuk pre-training model supervised.
* **Generative Models:** Menciptakan konten baru (gambar, musik, teks).

## **2. Efficient Data Representations dengan Autoencoders**

### **Konsep Dasar**
Autoencoder adalah jaringan saraf yang dilatih untuk merekonstruksi inputnya sendiri.
* **Encoder:** Mengubah input menjadi representasi laten (kode) yang lebih kecil.
* **Decoder:** Mengubah kode kembali menjadi input asli.

Kuncinya adalah **Bottleneck**: Kita membatasi ukuran kode (jumlah neuron di tengah) agar lebih kecil dari input. Ini memaksa jaringan untuk mempelajari pola-pola terpenting saja dan membuang *noise*.

### **Linear Autoencoder (PCA)**
Jika kita menggunakan fungsi aktivasi linear dan loss MSE, Autoencoder berperilaku persis seperti Principal Component Analysis (PCA).

Mari kita coba dengan dataset **Fashion MNIST**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Mempersiapkan Dataset Fashion MNIST
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]

# Flatten dataset untuk Dense Layer
X_train_flat = X_train.reshape(-1, 28 * 28)
X_valid_flat = X_valid.reshape(-1, 28 * 28)
X_test_flat = X_test.reshape(-1, 28 * 28)

# Membangun Stacked Autoencoder Sederhana
encoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[784]),
    keras.layers.Dense(30, activation="selu")  # Bottleneck layer (Latent Space)
])

decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(784, activation="sigmoid")
])

autoencoder = keras.models.Sequential([encoder, decoder])
autoencoder.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=1.5))

# Melatih Model (Autoencoder belajar merekonstruksi inputnya sendiri: X -> X)
history = autoencoder.fit(X_train_flat, X_train_flat, epochs=10, 
                          validation_data=(X_valid_flat, X_valid_flat), verbose=0)
print("Training Stacked Autoencoder selesai.")

### **Visualisasi Rekonstruksi**
Mari kita lihat seberapa baik Autoencoder mengompresi dan mengembalikan gambar baju.

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="binary")
    plt.axis("off")

def show_reconstructions(model, images=X_valid, n_images=5):
    reconstructions = model.predict(images.reshape(-1, 28 * 28))
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(images[image_index]) # Gambar Asli
        plt.title("Asli")
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index].reshape(28, 28)) # Rekonstruksi
        plt.title("Rekonstruksi")
    plt.show()

show_reconstructions(autoencoder)

### **Denoising Autoencoders**
Salah satu cara memaksa Autoencoder belajar fitur yang lebih robust adalah dengan menambahkan *noise* pada input, tetapi memintanya merekonstruksi gambar asli (bersih).
* Ini memaksa model untuk memisahkan sinyal (fitur struktur gambar) dari noise.

In [ ]:
# Menambahkan Gaussian Noise ke input
gaussian_noise = keras.layers.GaussianNoise(0.2)

denoising_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    gaussian_noise,
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu")
])

denoising_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

denoising_ae = keras.models.Sequential([denoising_encoder, denoising_decoder])
denoising_ae.compile(loss="binary_crossentropy", optimizer="adam")
# Note: Input ada noise, Target adalah gambar bersih (X_train)
# denoising_ae.fit(X_train, X_train, epochs=10, validation_data=(X_valid, X_valid))
print("Arsitektur Denoising Autoencoder siap.")
denoising_ae.summary()

## **3. Variational Autoencoders (VAE)**

Autoencoder biasa menghasilkan representasi laten yang deterministik. VAE berbeda: ia adalah **Probabilistic Generative Model**.

Alih-alih mempelajari satu vektor kode tetap, Encoder memprediksi **distribusi probabilitas** (Mean $\mu$ dan Log-Variance $\gamma$) dari mana kode akan disampling.

**Komponen Utama:**
1.  **Probabilistic Encoder:** Menghasilkan $\mu$ dan $\gamma$.
2.  **Sampling:** Mengambil sampel $z$ dari distribusi Gaussian menggunakan *Reparameterization Trick*.
3.  **Probabilistic Decoder:** Merekonstruksi gambar dari $z$.
4.  **Loss Function:** Gabungan dari *Reconstruction Loss* (agar mirip input) dan *Latent Loss* (KL Divergence, agar distribusi laten mendekati Normal Gaussian).

In [ ]:
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean

import tensorflow.keras.backend as K

codings_size = 10

# Encoder
inputs = keras.layers.Input(shape=[28, 28])
z = keras.layers.Flatten()(inputs)
z = keras.layers.Dense(150, activation="selu")(z)
z = keras.layers.Dense(100, activation="selu")(z)
codings_mean = keras.layers.Dense(codings_size)(z) # Mu
codings_log_var = keras.layers.Dense(codings_size)(z) # Gamma
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = keras.models.Model(
    inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

# Decoder
decoder_inputs = keras.layers.Input(shape=[codings_size])
x = keras.layers.Dense(100, activation="selu")(decoder_inputs)
x = keras.layers.Dense(150, activation="selu")(x)
x = keras.layers.Dense(28 * 28, activation="sigmoid")(x)
outputs = keras.layers.Reshape([28, 28])(x)
variational_decoder = keras.models.Model(inputs=[decoder_inputs], outputs=[outputs])

# VAE Model
_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
vae = keras.models.Model(inputs=[inputs], outputs=[reconstructions])

# Custom Loss (Reconstruction + KL Divergence)
latent_loss = -0.5 * K.sum(
    1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean),
    axis=-1)
vae.add_loss(K.mean(latent_loss) / 784.0)
vae.compile(loss="binary_crossentropy", optimizer="rmsprop")

print("Variational Autoencoder (VAE) siap dilatih untuk generative task.")

## **4. Generative Adversarial Networks (GANs)**

Diusulkan oleh Ian Goodfellow pada 2014, GAN adalah salah satu ide paling cemerlang di AI modern.

### **Konsep: The Adversarial Game**
GAN terdiri dari dua jaringan yang bersaing (seperti polisi vs pemalsu uang):
1.  **Generator:** Menerima *random noise* dan mencoba menghasilkan gambar palsu yang meyakinkan.
2.  **Discriminator:** Menerima gambar (asli dan palsu) dan mencoba menebak mana yang asli.

Tujuan Generator adalah menipu Discriminator. Tujuan Discriminator adalah tidak tertipu. Kompetisi ini mendorong keduanya menjadi semakin pintar hingga Generator menghasilkan data yang sangat realistis.

### **Pelatihan GAN**
Pelatihan GAN sangat sulit karena masalah **Nash Equilibrium** dan **Mode Collapse**. Kita harus melatih Discriminator satu langkah, lalu Generator satu langkah secara bergantian.

In [ ]:
codings_size = 30

# Generator: Noise -> Image
generator = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

# Discriminator: Image -> Real/Fake (Binary Classification)
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")
])

gan = keras.models.Sequential([generator, discriminator])

# Discriminator dilatih terpisah, jadi saat melatih GAN (Generator), bobot Discriminator dibekukan
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False 
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

print("Arsitektur Simple GAN berhasil didefinisikan.")

### **Training Loop GAN**
Berbeda dengan model biasa, kita tidak bisa hanya memanggil `.fit()`. Kita perlu *custom training loop*:
1.  Latih Discriminator dengan batch campuran (gambar asli + gambar palsu dari Generator).
2.  Latih Generator (lewat model `gan`) dengan memberikan label "Asli" pada gambar palsu (untuk menipu Discriminator).

In [ ]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=1):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        for X_batch in dataset:
            # Fase 1: Latih Discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size) # 0=Fake, 1=Real
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            
            # Fase 2: Latih Generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size) # Generator ingin Discriminator bilang ini "1" (Real)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
        print(f"Epoch {epoch+1} selesai.")

# Mempersiapkan dataset untuk training loop
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

# train_gan(gan, dataset, batch_size, codings_size) # Komentar: Pelatihan butuh waktu
print("Fungsi pelatihan GAN siap digunakan.")

## **5. Deep Convolutional GANs (DCGAN)**

GAN sederhana dengan Dense layer tidak bekerja baik untuk gambar kompleks. **DCGAN** menggunakan lapisan konvolusi (Conv2D) dan *Transposed Convolution* (untuk upsampling di Generator).

Panduan arsitektur DCGAN (dari paper asli Alec Radford):
* Gunakan `Strided Convolutions` untuk downsampling (Discriminator) dan `Transposed Convolutions` untuk upsampling (Generator).
* Gunakan `Batch Normalization` di kedua jaringan.
* Hindari `Dense` layers sebisa mungkin.
* Gunakan `ReLU` di Generator (kecuali output `Tanh`) dan `LeakyReLU` di Discriminator.

## **6. Kesimpulan**

Dalam Chapter 17 ini, kita telah menjelajahi sisi kreatif dari Deep Learning:
1.  **Autoencoders:** Alat yang ampuh untuk reduksi dimensi, *denoising*, dan mempelajari representasi data yang efisien tanpa label.
2.  **Variational Autoencoders (VAE):** Memperkenalkan konsep probabilistik ke dalam latent space, memungkinkan kita untuk men-generasi data baru dengan sampling.
3.  **GANs:** Membuka pintu untuk generasi konten hiper-realistis melalui kompetisi adversarial antara Generator dan Discriminator.

Pemahaman tentang representasi laten ini adalah kunci menuju konsep-konsep canggih seperti *Self-Supervised Learning* dan model generatif modern (seperti StyleGAN atau Stable Diffusion).